In [ ]:
!pip install langchain-cli

In [ ]:
!pip install -q langsmith

In [ ]:
%%capture
!pip install langchain langgraph langchain_core langchain_community langchain-google-genai

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import os

google_api_key=""

os.environ["GOOGLE_API_KEY"]=google_api_key

from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.2
)

In [ ]:
res=llm.invoke("what is skeleton of thought")

In [ ]:
res

AIMessage(content='"Skeleton of thought" isn\'t a formally defined term in any specific field like philosophy or psychology.  However, it\'s a useful metaphorical expression to describe the basic, underlying structure of an idea or argument.  It refers to the essential elements, stripped of all the supporting details, embellishments, and nuances.\n\nThink of it like this:\n\n* **The skeleton:** The bones provide the framework and support for the entire body.  Without them, the body collapses.\n* **The thought:** A fully fleshed-out idea, complete with examples, evidence, explanations, and stylistic flourishes.\n* **The skeleton of the thought:** The core argument, the main points, the logical progression – the bare minimum needed to convey the central idea.  It\'s the essential framework upon which the complete thought is built.\n\nHere\'s how it might be used:\n\n* **In brainstorming:**  You might start with the skeleton of an idea – a simple premise and a few key supporting points – 

In [ ]:
skeleton_generate_pormpt="""
[User:] You’re an organizer responsible for only giving the skeleton (not the full content) for answering the question.
Provide the skeleton in a list of points (numbered 1., 2., 3., etc.) to answer the question. Instead of writing a full sentence,
each skeleton point should be very short with only 3∼5 words. Generally, the skeleton should have 3∼10 points. Now,
please provide the skeleton for the following question.
{question}
Skeleton:
[Assistant:] 1
"""

In [ ]:
skleton_prompt=ChatPromptTemplate.from_template(skeleton_generate_pormpt)

skleton_chain=skleton_prompt | llm|StrOutputParser()

In [ ]:
question_example = "What are the benefits of exercise?"
skeleton_output = skleton_chain.invoke({"question": question_example})
print(skeleton_output)

1. Improved cardiovascular health
2. Weight management
3. Stronger muscles
4. Increased energy levels
5. Better mood
6. Reduced stress
7. Improved sleep
8. Stronger bones
9. Reduced disease risk


In [ ]:
expansion_temeplate="""
[User:] You’re responsible for continuing the writing of one and only one point in the overall answer to the following
question.
{question}
The skeleton of the answer is
{skeleton}
Continue and only continue the writing of point {point index}. Write it **very shortly** in 1∼2 sentence and
do not continue with other points!
[Assistant:] {point_index}. {point_skeleton}

"""
expansion_prompt=ChatPromptTemplate.from_template(expansion_temeplate)

expansion_chain=expansion_prompt | llm|StrOutputParser()

In [ ]:
question_example = "What are the benefits of exercise?"
final_output = expansion_chain.invoke({"question": question_example,"skeleton":skeleton_output,"point_index":'1',"point_skeleton":})


In [ ]:
final_output

In [ ]:
import re

def parse_numbered_list(numbered_list_string):
  """
  Parses a single numbered list string into a list of dictionaries.

  Args:
    numbered_list_string: A string containing a numbered list.

  Returns:
    A list of dictionaries, where each dictionary has keys 'index' and 'point'.
  """
  parsed_list = []
  # Use a regular expression to find numbered list items
  # It looks for a number followed by a dot, optional space, and then captures the rest of the line
  pattern = re.compile(r'^\s*(\d+)\.\s*(.*)$', re.MULTILINE)

  for match in pattern.finditer(numbered_list_string):
    index = int(match.group(1))
    point = match.group(2).strip()
    parsed_list.append({"index": index, "point": point})

  return parsed_list

# Example usage with the previous skeleton output
skeleton_output = """1. Improved cardiovascular health
2. Weight management
3. Stronger muscles
4. Increased energy levels
5. Better mood
6. Reduced stress
7. Improved sleep
8. Stronger bones
9. Reduced disease risk"""

parsed_skeleton = parse_numbered_list(skeleton_output)
print(parsed_skeleton)

[{'index': 1, 'point': 'Improved cardiovascular health'}, {'index': 2, 'point': 'Weight management'}, {'index': 3, 'point': 'Stronger muscles'}, {'index': 4, 'point': 'Increased energy levels'}, {'index': 5, 'point': 'Better mood'}, {'index': 6, 'point': 'Reduced stress'}, {'index': 7, 'point': 'Improved sleep'}, {'index': 8, 'point': 'Stronger bones'}, {'index': 9, 'point': 'Reduced disease risk'}]
